# Imports

In [1]:
from utils import *
import os
import shutil
from PIL import Image
from pytorch_fid import fid_score
import copy

import torch.distributions as D

import torch.optim.lr_scheduler as lrsc
import wandb
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

model_dict = dict()
model_dict['nf'] = 128
model_dict['ch_mult'] = (1, 2, 2)
model_dict['num_res_blocks'] = 2
model_dict['attn_resolutions'] = (16,)
model_dict['dropout'] = 0.1
model_dict['resamp_with_conv'] = True
model_dict['conditional'] = True
model_dict['nonlinearity'] = 'swish'
model_dict['sigma_max'] = 50
model_dict['sigma_min'] = 0.01
model_dict['num_scales'] = 1000
data_dict = dict()
data_dict['image_size'] = 32
data_dict['num_channels'] = 3
data_dict['centered'] = True
train_dict = dict()
train_dict['grad_clip'] = 1.0
train_dict['warmup'] = 5000
train_dict['lr'] = 2e-4
config_dict = dict()
config_dict['model'] = dotdict(model_dict)
config_dict['data'] = dotdict(data_dict)
config_dict['train'] = dotdict(train_dict)
config = dotdict(config_dict)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.login()
train_loader, val_loader = get_datasets()

wandb: Currently logged in as: vvoganesyan (use `wandb login --relogin` to force relogin)


Files already downloaded and verified
Files already downloaded and verified


# Sampling

In [2]:
eps_th = Unet(config)
eps_th.to(device)
ema_ = ema.ExponentialMovingAverage(eps_th.parameters(), decay=0.9999)

In [3]:
exp_name = 'ddpm_32x32'
eps_th.load_state_dict(torch.load(f'{exp_name}_model'), strict=False)
ema_.load_state_dict(torch.load(f'{exp_name}_ema'))

In [4]:
ema_.copy_to(eps_th.parameters())

# One model

In [5]:
with torch.no_grad():
    save_callable(lambda: sample_sde(device, eps_th), "data_3", 128)

In [15]:
save_dataloader(train_loader, 'data_1', 128*128)

In [13]:
res = fid_score.calculate_fid_given_paths(
        paths=['data_1', 'data_2'],
        batch_size=128,
        device=device,
        dims=2048
    )
print(res)

100%|██████████| 128/128 [00:34<00:00,  3.73it/s]


10.201483234328634


In [6]:
rmdir('data_1')

NameError: name 'shutil' is not defined